# **Entregable 5**
**🔧 Resumen de mejoras al código:**

**Documentación clara:**
Se agregaron docstrings a todas las funciones para explicar qué hacen, qué parámetros reciben y qué devuelven.

**Manejo de errores mejorado:**
Se añadieron verificaciones para evitar que el programa falle si hay errores en la conexión a la API o si los datos del CSV son inválidos.

**Validación de datos:**
Antes de analizar o graficar, se revisa que los datos estén completos y bien formateados (por ejemplo, que los números sean válidos).

**Código más limpio y organizado:**
Se reordenaron funciones, se eliminaron repeticiones y se usaron nombres más descriptivos para que el código sea más fácil de entender y mantener.

**Mensajes claros para el usuario:**
Se mejoraron los mensajes en consola para que sean más amigables y ayuden a entender qué está ocurriendo o si algo sale mal.

In [ ]:
!pip install requests
!pip install plotext
!pip install openpyxl

In [3]:
import requests
import csv
import json
from datetime import datetime
import statistics
import plotext as plt
from openpyxl import Workbook

API_KEY = "f0f0c07c117721896152aeca0fbbe31a"

def get_weather_current(city):
    """
    Consulta el clima actual de una ciudad desde la API de OpenWeather.

    Args:
        city (str): Nombre de la ciudad.

    Returns:
        dict or None: Diccionario con información del clima actual o None si falla la solicitud.
    """
    try:
        url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
        res = requests.get(url)
        if res.status_code != 200:
            print(f"❌ Error: Ciudad '{city}' no encontrada o API falló.")
            return None
        data = res.json()
        return {
            "fecha": datetime.utcfromtimestamp(data["dt"]).strftime("%Y-%m-%d"),
            "ciudad": data["name"],
            "temperatura": data["main"]["temp"],
            "humedad": data["main"]["humidity"],
            "presion": data["main"]["pressure"],
            "tipo": "actual"
        }
    except requests.exceptions.RequestException as e:
        print(f"❌ Error de red al consultar el clima: {e}")
        return None

def get_weather_forecast(city):
    """
    Consulta el pronóstico del clima a 5 días desde la API de OpenWeather.

    Args:
        city (str): Nombre de la ciudad.

    Returns:
        list: Lista de diccionarios con pronósticos diarios a las 18:00.
    """
    try:
        url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
        res = requests.get(url)
        if res.status_code != 200:
            print(f"❌ Error: Ciudad '{city}' no encontrada o API falló.")
            return []
        data = res.json()
        forecast_data = []
        for item in data["list"]:
            if item["dt_txt"].endswith("18:00:00"):
                forecast_data.append({
                    "fecha": item["dt_txt"].split()[0],
                    "ciudad": data["city"]["name"],
                    "temperatura": item["main"]["temp"],
                    "humedad": item["main"]["humidity"],
                    "presion": item["main"]["pressure"],
                    "tipo": "pronostico"
                })
        return forecast_data
    except requests.exceptions.RequestException as e:
        print(f"❌ Error de red al obtener el pronóstico: {e}")
        return []

def guardar_csv(datos, nombre_archivo):
    """
    Guarda datos en un archivo CSV.

    Args:
        datos (list): Lista de diccionarios con datos climáticos.
        nombre_archivo (str): Nombre del archivo CSV.
    """
    campos = ["fecha", "ciudad", "temperatura", "humedad", "presion", "tipo"]
    with open(nombre_archivo, mode='w', encoding='utf-8', newline='') as archivo:
        escritor = csv.DictWriter(archivo, fieldnames=campos)
        escritor.writeheader()
        escritor.writerows(datos)

def guardar_json(datos, nombre_archivo):
    """
    Guarda datos en un archivo JSON.

    Args:
        datos (list): Lista de diccionarios.
        nombre_archivo (str): Nombre del archivo JSON.
    """
    with open(nombre_archivo, mode='w', encoding='utf-8') as archivo:
        json.dump(datos, archivo, indent=4, ensure_ascii=False)

def guardar_excel(datos, nombre_archivo):
    """
    Guarda datos en un archivo Excel (.xlsx).

    Args:
        datos (list): Lista de diccionarios.
        nombre_archivo (str): Nombre del archivo Excel.
    """
    wb = Workbook()
    ws = wb.active
    ws.title = "Datos Clima"
    ws.append(["Fecha", "Ciudad", "Temperatura (°C)", "Humedad (%)", "Presión (hPa)", "Tipo"])
    for d in datos:
        ws.append([d["fecha"], d["ciudad"], d["temperatura"], d["humedad"], d["presion"], d["tipo"]])
    wb.save(nombre_archivo)

def leer_datos_csv(nombre_archivo):
    """
    Lee un archivo CSV y devuelve una lista de diccionarios con los datos.

    Args:
        nombre_archivo (str): Ruta al archivo CSV.

    Returns:
        list: Lista de diccionarios con datos validados.
    """
    datos = []
    try:
        with open(nombre_archivo, mode='r', encoding='utf-8') as archivo:
            lector = csv.DictReader(archivo)
            for fila in lector:
                try:
                    datos.append({
                        "fecha": fila["fecha"],
                        "ciudad": fila["ciudad"],
                        "temperatura": float(fila["temperatura"]),
                        "humedad": float(fila["humedad"]),
                        "presion": float(fila["presion"]),
                        "tipo": fila["tipo"]
                    })
                except ValueError:
                    print(f"⚠️ Datos inválidos en fila: {fila}")
    except FileNotFoundError:
        print(f"❌ Archivo CSV no encontrado: {nombre_archivo}")
    except Exception as e:
        print(f"❌ Error leyendo CSV: {e}")
    return datos

def analizar_datos(datos):
    """
    Realiza un análisis estadístico básico de los datos climáticos.

    Args:
        datos (list): Lista de diccionarios con datos.
    """
    if not datos:
        print("⚠️ No hay datos para analizar.")
        return

    # Validación adicional: eliminar entradas con datos faltantes
    datos = [d for d in datos if all(isinstance(d[k], (int, float)) for k in ["temperatura", "humedad", "presion"])]
    if not datos:
        print("⚠️ Todos los datos eran inválidos o incompletos.")
        return

    temperaturas = [d["temperatura"] for d in datos]
    humedades = [d["humedad"] for d in datos]
    presiones = [d["presion"] for d in datos]

    print("\n📈 Resultados del análisis:")
    print(f"🌡️ Temperatura promedio: {statistics.mean(temperaturas):.2f} °C")
    if len(temperaturas) > 1:
        print(f"📉 Desviación estándar de la temperatura: {statistics.stdev(temperaturas):.2f} °C")
    else:
        print("⚠️ No hay suficientes datos para calcular la desviación estándar.")
    print(f"💧 Humedad promedio: {statistics.mean(humedades):.2f} %")
    print(f"🧭 Rango de presión atmosférica: {min(presiones):.2f} - {max(presiones):.2f} hPa")
    print(f"🏙️ Ciudades analizadas: {', '.join(set(d['ciudad'] for d in datos))}")
    print(f"📅 Días analizados: {', '.join(set(d['fecha'] for d in datos))}")

def graficar_lineas(datos):
    """
    Genera un gráfico de línea de la temperatura promedio por día.
    """
    fechas = sorted(set(d["fecha"] for d in datos))
    fechas_convertidas = [datetime.strptime(f, "%Y-%m-%d").strftime("%d/%m/%Y") for f in fechas]
    temperaturas = [sum(d["temperatura"] for d in datos if d["fecha"] == f) / len([d for d in datos if d["fecha"] == f]) for f in fechas]

    plt.clear_figure()
    plt.title("🌡️ Temperatura promedio diaria")
    plt.plot(fechas_convertidas, temperaturas, marker='dot')
    plt.xlabel("Fecha (DD/MM/YYYY)")
    plt.ylabel("Temperatura (°C)")
    plt.grid(True)
    plt.show()

def graficar_barras(datos):
    """
    Genera un gráfico de barras de la humedad promedio por día.
    """
    fechas = sorted(set(d["fecha"] for d in datos))
    fechas_convertidas = [datetime.strptime(f, "%Y-%m-%d").strftime("%d/%m/%Y") for f in fechas]
    humedades = [sum(d["humedad"] for d in datos if d["fecha"] == f) / len([d for d in datos if d["fecha"] == f]) for f in fechas]

    plt.clear_figure()
    plt.title("💧 Humedad promedio diaria")
    plt.bar(fechas_convertidas, humedades, color="cyan")
    plt.xlabel("Fecha (DD/MM/YYYY)")
    plt.ylabel("Humedad (%)")
    plt.grid(True)
    plt.show()

def graficar_dispersion(datos):
    """
    Muestra un gráfico de dispersión entre temperatura y presión.
    """
    temperaturas = [d["temperatura"] for d in datos]
    presiones = [d["presion"] for d in datos]

    plt.clear_figure()
    plt.title("🧭 Presión vs Temperatura")
    plt.scatter(temperaturas, presiones)
    plt.xlabel("Temperatura (°C)")
    plt.ylabel("Presión (hPa)")
    plt.grid(True)
    plt.show()

def menu_graficas(datos):
    """
    Menú interactivo para seleccionar y mostrar gráficos.
    """
    while True:
        print("\n📊 Elige una opción para visualizar:")
        print("1. Gráfico de líneas (Temperatura)")
        print("2. Gráfico de barras (Humedad)")
        print("3. Diagrama de dispersión (Presión vs Temperatura)")
        print("4. Ver todas")
        print("5. Salir")

        opcion = input("👉 Tu elección: ").strip()
        if opcion == "1":
            graficar_lineas(datos)
        elif opcion == "2":
            graficar_barras(datos)
        elif opcion == "3":
            graficar_dispersion(datos)
        elif opcion == "4":
            graficar_lineas(datos)
            graficar_barras(datos)
            graficar_dispersion(datos)
        elif opcion == "5":
            break
        else:
            print("⚠️ Opción inválida. Intenta de nuevo.")

def mostrar_datos(actual, pronostico):
    """
    Imprime en consola los datos climáticos actuales y del pronóstico.

    Args:
        actual (dict): Datos actuales.
        pronostico (list): Lista de datos del pronóstico.
    """
    print(f"\n🌆 Clima actual en {actual['ciudad']} ({actual['fecha']}):")
    print(f"  Temperatura: {actual['temperatura']} °C")
    print(f"  Humedad: {actual['humedad']} %")
    print(f"  Presión: {actual['presion']} hPa")
    print("\n📅 Pronóstico a 5 días (18:00 hrs):")
    for p in pronostico:
        print(f"  {p['fecha']} -> Temp: {p['temperatura']} °C, Humedad: {p['humedad']} %, Presión: {p['presion']} hPa")

def main():
    """
    Función principal que coordina el flujo completo del programa.
    """
    ciudad = input("🌆 Ingrese la ciudad para consultar el clima: ").strip()
    if not ciudad:
        print("❌ No ingresó ninguna ciudad.")
        return

    actual = get_weather_current(ciudad)
    if actual is None:
        return

    pronostico = get_weather_forecast(ciudad)
    mostrar_datos(actual, pronostico)

    datos_guardar = [actual] + pronostico
    guardar_csv(datos_guardar, "clima_datos.csv")
    guardar_json(datos_guardar, "clima_datos.json")
    guardar_excel(datos_guardar, "clima_datos.xlsx")

    print("\n✅ Datos guardados en 'clima_datos.csv', 'clima_datos.json' y 'clima_datos.xlsx'")

    datos_csv = leer_datos_csv("clima_datos.csv")
    print("\n--- Análisis basado en CSV ---")
    analizar_datos(datos_csv)
    menu_graficas(datos_csv)

if __name__ == "__main__":
    main()


🌆 Ingrese la ciudad para consultar el clima: Mexicali

🌆 Clima actual en Mexicali (2025-05-16):
  Temperatura: 26.72 °C
  Humedad: 24 %
  Presión: 1010 hPa

📅 Pronóstico a 5 días (18:00 hrs):
  2025-05-16 -> Temp: 27.52 °C, Humedad: 19 %, Presión: 1010 hPa
  2025-05-17 -> Temp: 28.22 °C, Humedad: 20 %, Presión: 1009 hPa
  2025-05-18 -> Temp: 24.74 °C, Humedad: 23 %, Presión: 1011 hPa
  2025-05-19 -> Temp: 28.22 °C, Humedad: 24 %, Presión: 1012 hPa
  2025-05-20 -> Temp: 33.4 °C, Humedad: 7 %, Presión: 1013 hPa

✅ Datos guardados en 'clima_datos.csv', 'clima_datos.json' y 'clima_datos.xlsx'

--- Análisis basado en CSV ---

📈 Resultados del análisis:
🌡️ Temperatura promedio: 28.14 °C
📉 Desviación estándar de la temperatura: 2.89 °C
💧 Humedad promedio: 19.50 %
🧭 Rango de presión atmosférica: 1009.00 - 1013.00 hPa
🏙️ Ciudades analizadas: Mexicali
📅 Días analizados: 2025-05-18, 2025-05-17, 2025-05-16, 2025-05-19, 2025-05-20

📊 Elige una opción para visualizar:
1. Gráfico de líneas (Temperatu

Los datos se guardan correctamente en documentos .csv y .json así como en una tabla ordenada en una hoja de excel de maneta automática.